In [ ]:
'''
credits:
jupyter notebooks: Krish Naik, Matthew Berman
                 :
                 :
datasets:
        : https://www.kaggle.com/datasets/ratthachat/writing-prompts
        : https://huggingface.co/datasets/karmat314/writingprompts-story
'''

In [ ]:
# import tensorflow as tf
# # tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster("/device:GPU:0")
# tf.tpu.experimental.initialize_tpu_system("/device:GPU:0")
# tpu_strategy = tf.distribute.experimental.TPUStrategy("/device:GPU:0")

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     print("Name:", gpu.name, "  Type:", gpu.device_type)
    
# from tensorflow.python.client import device_lib

# device_lib.list_local_devices()

# tf.test.is_gpu_available()

#**Step 1: Install All the Required Packages**

In [4]:
#run this if using google colab. If running locally, install everything manually
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 scikit-learn

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


#**Step 2: Import All the Required Libraries**

QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-storyteller-v2-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64 # rank of the matrix.

# Alpha parameter for LoRA scaling
lora_alpha = 16 # Scaling factor. Controls the magnitude of update.

# Dropout probability for LoRA layers
lora_dropout = 0.1 # random droppping to prevent overfitting.

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True # 32 bit parameters are loaded as 4 bit (quantization)
# use_4bit = False # for cpu training. (In case of no gpu)

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4) (Quantization method)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.01

# Optimizer to use
optim = "paged_adamw_32bit"
# optim = "adamw" # a common optimizer

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 200

# Log every X updates steps
logging_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}
# device_map = "auto"

#**Step 4:Load everything and start the fine-tuning process**

1. First of all, we want to load the dataset we defined. Here, our dataset is already preprocessed but, usually, this is where you would reformat the prompt, filter out bad text, combine multiple datasets, etc.


2. Then, we’re configuring bitsandbytes for 4-bit quantization.


3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.


4. Finally, we're loading configurations for QLoRA, regular training parameters, and passing everything to the SFTTrainer. The training can finally start!

In [ ]:
#use only if running on google colab
from google.colab import drive
drive.mount('/content/drive/')

In [6]:
import re

system_prompt = "You are a creative writer. Generate engaging short stories based on the provided prompts. Each story should have a clear beginning, middle, and end, and should be imaginative and well-written."
pattern = r'\[.*?\]\s*'
prompt_rows = []
story_rows = []
avg_story_length = 0

In [7]:

print("Parsing data...")
# path_to_prompts = "/content/drive/My Drive/Storyteller/dataset/writingPrompts/test.wp_source" # use this path if running on colab
# path_to_stories = "/content/drive/My Drive/Storyteller/dataset/writingPrompts/test.wp_target" # use this path if running on colab
path_to_prompts = "test_source.txt"  # use this path if running locally
path_to_stories ="test_target.txt"    # use this path if running locally
path_to_prompts = "/kaggle/input/15k-story-wala-dataset/test_source.txt"
path_to_stories = "/kaggle/input/15k-story-wala-dataset/test_target.txt"
with open(path_to_prompts,'r', encoding="utf-8") as f:
  for line in f:
    cleaned_line = re.sub(pattern, '', line).strip()
    prompt_rows.append(cleaned_line)

with open(path_to_stories, 'r', encoding="utf-8") as f:
  for line in f:
    avg_story_length+=len(line)
    story_rows.append(line.strip())
# prompt_rows = prompt_rows[:len(story_rows)]
print("Parsing Completed")

Parsing data...
Parsing Completed


In [8]:
# avg_story_length/=len(story_rows)
# print(avg_story_length)
prompt_rows = prompt_rows[1000:4000]
story_rows = story_rows[1000:4000]

In [9]:
print(prompt_rows[0])
print(story_rows[0])
print(len(prompt_rows), len(story_rows))
print(prompt_rows[975])
print(story_rows[975])

Dragons are discovered , but few people can train them . A redditor trains one to violently dispatch people who make up shitty writing prompts .
I struggled in vain , and the typos remian , because how dare I edit a comment in reddit . I do n't try to hard and prepare in advance but today i wish I had brought a lance . Because , you see a dragon did come and ne'er before have I thought to run . But no despite the obvious sleight , i stood up in righteousness and elected to fight . The dragon did struggle and spit and rebel , but really , is this a challenge , or just some fresh hell . A protest in jest or a laughable jape . Intent barely hidden by the will of an ape , mocking the efforts we all do put forth , and yet , checking closely , His efforts are naught . We measure by voting and by laughing appeal , flocking to fame with vim and some zeal . I realise I 've overstepped by a yard , but jesus , this website , formatting is hard .
3000 3000
Humans have five senses , while every oth

In [ ]:
# no need to run
import pandas as pd
# path = "/content/drive/My Drive/train-00000-of-00001.parquet"
local_path = "train-00000-of-00001.parquet"
kag_path = "/kaggle/input/400-stories-and-prompt-dataset"
df = pd.read_parquet(kag_path) # use local_path if running locally

# df.head()
# df["prompt"][0]
# len(df["story"])
for prompt, story in zip(df["prompt"], df["story"]):
    cleaned_line = re.sub(pattern, '', prompt).strip()
    prompt_rows.append(cleaned_line)
    cleaned_story = re.sub(pattern, '', story).strip()
    avg_story_length+=len(cleaned_story)
    story_rows.append(cleaned_story)

In [10]:
# Forming the dataset
data_list = []
start_line = f"<s> [INST] <<SYS>> \n {system_prompt} \n <</SYS>> \n \n"
# data_available_till_line = 9751
for prompt, story in zip(prompt_rows, story_rows):
  end_line = f"{prompt} [/INST] {story} </s>"
  full_line = start_line + end_line
  data_list.append(full_line)

In [11]:
print(len(data_list))
data_list[800]

3000


"<s> [INST] <<SYS>> \n You are a creative writer. Generate engaging short stories based on the provided prompts. Each story should have a clear beginning, middle, and end, and should be imaginative and well-written. \n <</SYS>> \n \nWrite the last paragraph of a science fiction epic . [/INST] Cintis watched as the gleaming pearlescent spires were swallowed by the crumbling ocean . Any utopia built on human suffering did n't deserve its preservation . <newline> <newline> A Spacial Rend opened silently next to him and Isa stepped out , surveying he damage . Her artificial eye gleamed , like current being run through an emerald . `` I see you removed the Mutagenix Lab . How many casualties ? '' she asked cooly . <newline> <newline> `` Less than one hundred . It does n't matter . We 're safe now . '' Isa 's genetically customized features hopefully looked up at Cintis . He admired her pale features , lifting off her good and running his finger across her GeneFuse seam . He stepped through 

In [12]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
# data_dict_list = [{"text": text} for text in data_list]
# dataset = Dataset.from_list(data_dict_list)
# print(dataset[0])

# Split the dataset into training, validation, and test sets
train_data, temp_data = train_test_split(data_list, test_size=0.2, random_state=41)
print("lengths:",len(train_data),len(temp_data))
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=41)

lengths: 2400 600


In [13]:
print(len(val_data), len(test_data))

300 300


In [14]:
train_data_dict = [{"text": text} for text in train_data]
# train_data_dict = train_data_dict[:4000]
train_data = Dataset.from_list(train_data_dict)

test_data_dict = [{"text": text} for text in test_data]
# test_data_dict = test_data_dict[:800]
test_data = Dataset.from_list(test_data_dict)

val_data_dict = [{"text": text} for text in val_data]
# val_data_dict = val_data_dict[:800]
val_data = Dataset.from_list(val_data_dict)

In [16]:
train_data[0]

{'text': "<s> [INST] <<SYS>> \n You are a creative writer. Generate engaging short stories based on the provided prompts. Each story should have a clear beginning, middle, and end, and should be imaginative and well-written. \n <</SYS>> \n \nA veteran Holocaust Jewish survivor meets a man in the elevator . As they strike a conversation , he slowly realizes the man was his brutal Nazi prison warden . [/INST] How we both ended up on an elevator in America , I can never be quite sure . It had been years since the war , and the appeal of the states had drawn me from my war-torn home in Warsaw . I had noticed the man briefly before entering the elevator . We were both on the top floor of the skyscraper , looking out over the pristine American city . As the sun finally set , we both exhaled softly and headed toward the elevator doors . <newline> <newline> We both stepped in , and the doors closed us in together . I stayed to one side , as far away from the man as I could . I 'd never been ke

In [15]:
print(type(train_data))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
def get_last_checkpoint(output_dir):
    checkpoints = [d for d in os.listdir(output_dir) if re.match(r"checkpoint-\d+", d)]
    if not checkpoints:
        return None
    checkpoints = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))
    return os.path.join(output_dir, checkpoints[-1])

In [17]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")
# from transformers import get_last_checkpoint
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability() #torch.cuda.get-- changed ro torch.get--
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

#checkpointing
# output_direc = "./model_checkpoints"

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # load_best_model_at_end=True,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    eval_dataset=val_data,
    # eval_test=test_data
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [18]:


# Check if a checkpoint exists and resume from it if available
# last_checkpoint = None
# if os.path.isdir(output_dir):
#     last_checkpoint = get_last_checkpoint(output_dir)
#     if last_checkpoint is not None:
#         print(f"Resuming from checkpoint: {last_checkpoint}")

# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,2.258900
200,1.942500
300,1.940100
400,1.959200
500,1.982100
600,1.940500
700,1.943300
800,1.915600
900,1.965900
1000,1.933000


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain curr

TrainOutput(global_step=2400, training_loss=1.9285925229390461, metrics={'train_runtime': 16300.01, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.147, 'total_flos': 7.550695403569152e+16, 'train_loss': 1.9285925229390461, 'epoch': 2.0})

In [27]:
torch.cuda.empty_cache()

In [23]:
# Save trained model
trainer.model.save_pretrained(new_model)

##**Step 5: Check the plots on tensorboard, as follows**

In [20]:
%load_ext tensorboard
%tensorboard --logdir results/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4907), started 0:00:43 ago. (Use '!kill 4907' to kill it.)

In [21]:
model.save_pretrained('./manual_save')
tokenizer.save_pretrained('./manual_save')

NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported

In [ ]:
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
prompt = "write a creative story on the following topic: Two undercover cops try to arrest each other."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] \n {prompt} [/INST]")
print(result[0]['generated_text'])

###**Step 6:Use the text generation pipeline to ask questions like “What is a large language model?” Note that I’m formatting the input to match Llama 2 prompt template.**

In [24]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "A story about hare and tortoise."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] <<SYS>> \n {system_prompt} \n <</SYS>> \n \n {prompt} [/INST]")
print(result[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will

<s>[INST] <<SYS>> 
 You are a creative writer. Generate engaging short stories based on the provided prompts. Each story should have a clear beginning, middle, and end, and should be imaginative and well-written. 
 <</SYS>> 
 
 A story about hare and tortoise. [/INST] The hare was a bit of a show-off. He was always bragging about how fast he could run, and how he could beat anyone in a race. <newline> <newline> One day, he decided to challenge the tortoise to a race. The tortoise was a bit of a slowpoke, but he was determined to prove the hare wrong. <newline> <newline> The race began, and the hare took off like a shot. He was so fast that he was almost out of sight in a matter of seconds. <newline> <newline> The tortoise, however, was a bit slower. He wasn't trying to win, he just wanted to prove that he could keep up with the hare. <newline> <newline> As the race continued, the hare began to tire. He was running so fast that he was getting tired, and he was starting to lose his breat

In [31]:
# Empty VRAM
import gc
del model
del pipeline
del trainer

gc.collect()
gc.collect()

NameError: name 'model' is not defined

You can train a Llama 2 model on the entire dataset using [mlabonne/guanaco-llama2](https://huggingface.co/datasets/mlabonne/guanaco-llama2)

#**Step 7: Store New Llama2 Model (Llama-2-7b-chat-finetune)**

How can we store our new Llama-2-7b-chat-finetune model now? We need to merge the weights from LoRA with the base model. Unfortunately, as far as I know, there is no straightforward way to do it: we need to reload the base model in FP16 precision and use the peft library to merge everything.

In [35]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
#     load_in_8bit = True,
    device_map="cpu",
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [37]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [41]:
from huggingface_hub import create_repo
# Create a new repository
repo_name = "Llama-2-7b-storyteller-v2-finetune"  # Replace with the desired repo name
username = "kulkarni-atharva"    # Replace with your username
repo_id = f"{username}/{repo_name}"

create_repo(repo_id, exist_ok=True) 

RepoUrl('https://huggingface.co/kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune')

In [40]:
!pip install -q huggingface_hub
from huggingface_hub import login
login(token="token_hidden")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [42]:


model.push_to_hub("kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune", check_pr=True)

tokenizer.push_to_hub("kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune",check_pr=True)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune/commit/afe72456677cdeafb8100ae967cb42c8d2074aaf', commit_message='Upload tokenizer', commit_description='', oid='afe72456677cdeafb8100ae967cb42c8d2074aaf', pr_url=None, pr_revision=None, pr_num=None)

#**Step 8: Push Model to Hugging Face Hub**

Our weights are merged and we reloaded the tokenizer. We can now push everything to the Hugging Face Hub to save our model.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!huggingface-cli login

model.push_to_hub("kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune", check_pr=True)

tokenizer.push_to_hub("kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune",check_pr=True)


You can now use this model for inference by loading it like any other Llama 2 model from the Hub.